Importing the relevant libraries

In [ ]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pickle
import tensorflow as tf
import matplotlib.pyplot as plt
import pathlib
import os

Lyon cochlear model to convert soundwaves to cochlear activations

In [ ]:
pip install lyon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install pyspikes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from numpy import float64
from lyon.calc import LyonCalc
from librosa import load
from spikes import encoder

In [ ]:
seed=42
tf.random.set_seed(seed)
np.random.seed(seed)

In [ ]:
DATASET_PATH = 'data/mini_speech_commands'
data_dir = pathlib.Path(DATASET_PATH)
if not data_dir.exists():
  tf.keras.utils.get_file(
      'mini_speech_commands.zip',
      origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
      extract=True,
      cache_dir='.', cache_subdir='data')

182091776/182082353 [==============================] - 3s 0us/step


In [ ]:
commands = np.array(tf.io.gfile.listdir(str(data_dir)))
commands = commands[commands != 'README.md']
print('Commands:', commands)

Commands: ['left' 'down' 'no' 'yes' 'stop' 'right' 'go' 'up']


In [ ]:
x_train=[]
y_train=[]

for label in commands:
  print(label)
  for filename in os.listdir(os.path.join(DATASET_PATH,label)):
    if (not(filename.endswith('.wav'))):
      continue
    waveform,sample_rate=load(os.path.join(DATASET_PATH,label,filename))
    calc = LyonCalc()
    decimation_factor = 16
    coch = calc.lyon_passive_ear(waveform.astype(float64), sample_rate, decimation_factor)
    samples=np.reshape(coch,[1,coch.shape[0],coch.shape[1]])
    if np.min(samples)<0:
      print(np.min(samples))
    bsa = encoder.BSA(samples)
    spikes = np.int8(bsa.get_spikes())
    x_train.append(spikes)
    y_train.append(label)

left
down
no
yes
stop
right
go
up


In [ ]:
print(len(x_train))
print(x_train[0].shape)
print(x_train[0].dtype)

8000
(1, 2205, 96)
int8


In [ ]:
m=max([x.shape[1] for x in x_train])
x_train=[np.reshape(np.concatenate((x[0],np.zeros((m-x.shape[1],x.shape[2])))),(x.shape[0],-1,x.shape[2])) for x in x_train]

In [ ]:
import random
temp=list(zip(x_train,y_train))
random.shuffle(temp)
x_train,y_train=zip(*temp)
x_train=list(x_train)
y_train=list(y_train)

In [ ]:
x_data=np.int8(np.concatenate(x_train))
y_data=np.array(y_train)

In [ ]:
print(x_data.shape)
print(y_data.shape)

(8000, 1378, 96)
(8000,)


In [ ]:
print(x_data.dtype)

int8


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_path = '/content/drive/My Drive/Colab Notebooks/tf_LSM/'
np.save(data_path + 'mini_speech_data_decimation_16', x_data)
np.save(data_path + 'mini_speech_label_decimation_16', y_data)

In [ ]:
test_load_data = np.load(data_path + 'mini_speech_data_decimation_16.npy')
test_load_label = np.load(data_path + 'mini_speech_label_decimation_16.npy')
print(np.max(test_load_data - x_data))
print(np.min(test_load_data - x_data))

0
0


In [ ]:
for i in range(100):
  print('loaded label i: ', test_load_label[i])
  print('original label i: ', y_data[i])


loaded label i:  go
original label i:  go
loaded label i:  go
original label i:  go
loaded label i:  down
original label i:  down
loaded label i:  up
original label i:  up
loaded label i:  down
original label i:  down
loaded label i:  up
original label i:  up
loaded label i:  left
original label i:  left
loaded label i:  down
original label i:  down
loaded label i:  no
original label i:  no
loaded label i:  up
original label i:  up
loaded label i:  go
original label i:  go
loaded label i:  down
original label i:  down
loaded label i:  left
original label i:  left
loaded label i:  stop
original label i:  stop
loaded label i:  left
original label i:  left
loaded label i:  down
original label i:  down
loaded label i:  go
original label i:  go
loaded label i:  down
original label i:  down
loaded label i:  left
original label i:  left
loaded label i:  up
original label i:  up
loaded label i:  go
original label i:  go
loaded label i:  go
original label i:  go
loaded label i:  no
original lab

In [ ]:
'''
data_size_test = x_train.shape[0]//5

x_test=np.array(x_train[-data_size_test:])
y_test=np.array(y_train[-data_size_test:])
x_in_train=np.array(x_train[:-data_size_test])
y_in_train=np.array(y_train[:-data_size_test])

data_size = x_in_train.shape[0]
'''